# **Predicción de Tiempo de Maratón Basado en Actividades de Entrenamiento**
 
## **Descripción**
Este notebook construye un modelo de regresión para predecir el tiempo de maratón (`MarathonTime`) usando diversos parámetros de entrenamiento.

In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import re

# Cargar el dataset
try:
    data = pd.read_csv('MarathonData.csv')  # Reemplaza 'ruta_al_archivo.csv' con la ruta correcta
    print("Dataset cargado exitosamente.")
except FileNotFoundError as e:
    print("Error: Asegúrate de que el archivo CSV esté en el directorio correcto.")
    raise e


Dataset cargado exitosamente.


In [35]:
# Exploración del Dataset
print("\n**Primeras 5 filas del Dataset de Actividades:**")
display(data)


**Primeras 5 filas del Dataset de Actividades:**


,id,Marathon,Name,Category,km4week,sp4week,CrossTraining,Wall21,MarathonTime,CATEGORY
0,1,Prague17,Blair MORGAN,MAM,132.8,14.434783,NaN,1.16,2.37,A
1,2,Prague17,Robert Heczko,MAM,68.6,13.674419,NaN,1.23,2.59,A
2,3,Prague17,Michon Jerome,MAM,82.7,13.520436,NaN,1.30,2.66,A
3,4,Prague17,Daniel Or lek,M45,137.5,12.258544,NaN,1.32,2.68,A
4,5,Prague17,Luk ? Mr zek,MAM,84.6,13.945055,NaN,1.36,2.74,A
...,...,...,...,...,...,...,...,...,...,...
82,83,Prague17,Stefano Vegliani,M55,50.0,10.830325,NaN,2.02,3.93,D
83,84,Prague17,Andrej Madliak,M40,33.6,10.130653,ciclista 3h,1.94,3.93,D
84,85,Prague17,Yoi Ohsako,M40,55.4,11.043189,NaN,1.94,3.94,D
85,86,Prague17,Simon Dunn,M45,33.2,11.066667,NaN,2.05,3.95,D


## **Preprocesamiento de los Datos**

In [36]:
# Convertir 'MarathonTime' a numérico, forzando errores a NaN, y eliminarlos después
data['MarathonTime'] = pd.to_numeric(data['MarathonTime'], errors='coerce')
data['km4week'] = pd.to_numeric(data['km4week'], errors='coerce')
data['sp4week'] = pd.to_numeric(data['sp4week'], errors='coerce')
data['Wall21'] = pd.to_numeric(data['Wall21'], errors='coerce')
data = data.dropna(subset=['MarathonTime', 'km4week', 'sp4week', 'Wall21'])

# Procesamiento de 'CrossTraining' (extraer horas)
def extract_hours(cross_training):
    if pd.isnull(cross_training):
        return 0.0  # Asignar 0 si el valor es NaN
    match = re.search(r'(\d+\.?\d*)\s*h', cross_training.lower())
    if match:
        return float(match.group(1))
    return 0.0  # Asignar 0 si no se encuentra un patrón de horas

data['CrossTraining_Hours'] = data['CrossTraining'].apply(extract_hours)

# Convertir variables categóricas en variables dummy
data = pd.get_dummies(data, columns=['Category'], drop_first=True)

# Eliminar columnas que no se usarán en el modelo
data = data.drop(columns=['id', 'Marathon', 'Name', 'CrossTraining', 'CATEGORY'])

## **División de Datos para Entrenamiento y Pruebas**

In [37]:
# Definir variables independientes (X) y variable dependiente (y)
X = data.drop(columns=['MarathonTime'])
y = data['MarathonTime']

# División del conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## **Entrenamiento del Modelo de Regresión**

In [38]:
# Inicializar el modelo de regresión lineal
model = LinearRegression()

# Entrenar el modelo
model.fit(X_train, y_train)

LinearRegression()

## **Evaluación del Modelo**

In [39]:
# Predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular métricas de rendimiento
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\n**MSE (Error Cuadrático Medio):** {mse:.2f}")
print(f"**R^2 (Coeficiente de Determinación):** {r2:.2f}")


**MSE (Error Cuadrático Medio):** 0.01
**R^2 (Coeficiente de Determinación):** 0.94


## **Predicción de Tiempo de Maratón para Nuevas Actividades**

In [40]:
# Definir una función para predecir el tiempo de maratón en base a una nueva actividad
def predecir_tiempo_maraton(nueva_actividad):
    """
    Predice el tiempo de maratón basado en una nueva actividad.
    
    Parámetros:
    - nueva_actividad: Diccionario con detalles de la actividad (distancia semanal, velocidad semanal, horas de cross-training, etc.)
    
    Retorna:
    - tiempo_predicho: Predicción de tiempo de maratón.
    """
    # Convertir el diccionario en un DataFrame
    nueva_actividad_df = pd.DataFrame([nueva_actividad])
    
    # Convertir 'Category' a dummies
    nueva_actividad_df = pd.get_dummies(nueva_actividad_df, columns=['Category'], drop_first=True)
    
    # Alinear columnas con las de X (agregar columnas faltantes con valor 0)
    nueva_actividad_df = nueva_actividad_df.reindex(columns=X.columns, fill_value=0)
    
    # Realizar la predicción
    tiempo_predicho = model.predict(nueva_actividad_df)[0]
    
    return tiempo_predicho

## **Ejemplo de Predicción**

In [42]:
# Datos de una nueva actividad para predicción
nueva_actividad = {
    'km4week': 80.0,
    'sp4week': 12.5,
    'Wall21': 1.5,
    'CrossTraining_Hours': 2,  # Ajustar según la actividad
    'Category': 1,  # Cambiar según la categoría de la nueva actividad
}

# Realizar la predicción
tiempo_predicho = predecir_tiempo_maraton(nueva_actividad)
print(f"\n**Tiempo Predicho para Maratón Basado en Nueva Actividad:** {tiempo_predicho:.2f} horas")


**Tiempo Predicho para Maratón Basado en Nueva Actividad:** 3.04 horas


## **Conclusión**

Hemos implementado un modelo de regresión lineal que predice el tiempo total de maratón usando información sobre entrenamiento semanal y otros parámetros.

---